In [1]:
#!conda install --yes -n cropmodel -c conda-forge cppyy

In [2]:
import cppyy
from ctypes import c_double, c_long, c_int

In [3]:
cppyy.include('../server/model/TGRAINS.h')
cppyy.load_library('../server/model/libTGRAINS.so')

In [4]:
from cppyy.gbl.std import vector

cropAreas = vector['double'](11)
livestockAreas = vector['double'](6)
myUniqueLandscapeID = c_int(101)
maxCropArea = c_double(0)
error = c_int(0)

In [33]:
cppyy.gbl.initialise(101, maxCropArea, cropAreas, livestockAreas, error)
cropAreas.size()

11

In [34]:
#void run(
#    vector<double> cropAreas
#    vector<double> livestockAreas
#    double& greenhouseGasEmissions
#    double& nLeach
#    vector<double>& pesticideImpacts
#    double& profit
#    double& production
#    vector<double>& nutritionaldelivery
#    vector<double>& healthRiskFactors
#    int& errorFlag);

# cropAreas defined above
# livestockAreas defined above
# error (flag) defined above
greenhouseGasEmissions = c_double(0)
nLeach                 = c_double(0)
pesticideImpacts       = vector['double'](range(5))
profit                 = c_double(0)
production             = c_double(0)
nutritionaldelivery    = vector['double']()
healthRiskFactors      = vector['double']()

In [35]:
maxCropArea

c_double(363766.22727000003)

In [39]:
for i in range(0,cropAreas.size()):
    print(cppyy.gbl.getCropString(i), '\t', cropAreas[i])

FRUIT_SOFT 	 1673.3372840000002
ALL_PEAS_BEANS 	 13387.184789395684
ALL_OATS 	 6037.01735577993
SUGARBEET 	 23171.312951484382
ALL_BARLEY 	 49777.655426557394
MAIZE 	 9796.96465172778
ALL_WHEAT 	 51257.81372481785
ALL_VEGETABLES 	 21687.045396768444
OSR_WINTER 	 32646.197396086704
ALL_POTATO 	 12936.45407116927
FRUIT_TOP 	 1909.0199859999998


In [40]:
cppyy.gbl.getLandscapeString(101)

'East Anglia Study Area'

In [41]:
totalCropArea = 0
for d in cropAreas:
    totalCropArea = totalCropArea + d
totalCropArea

224280.00303378748

In [42]:
cropProps = vector['double']((0.1, 0.05, 0.1, 0.075, 0.125, 0.07, 0.06, 0.05, 0.05, 0.025, 0.05))
for i in range(cropAreas.size()):
    cropAreas[i] = totalCropArea * cropProps.at(i)

In [43]:
import time
start = time.time()

In [44]:
print([i for i in cropAreas])

[22428.00030337875, 11214.000151689375, 22428.00030337875, 16821.00022753406, 28035.000379223435, 15699.600212365125, 13456.800182027248, 11214.000151689375, 11214.000151689375, 5607.000075844688, 11214.000151689375]


In [45]:
# Run model
cppyy.gbl.run(
    cropAreas,
    livestockAreas,
    greenhouseGasEmissions,
    nLeach,
    pesticideImpacts,
    profit,
    production,
    nutritionaldelivery,
    healthRiskFactors,
    error
)

length_error: void ::run(vector<double> cropAreas, vector<double> livestockAreas, double& greenhouseGasEmissions, double& nLeach, vector<double>& pesticideImpacts, double& profit, double& production, vector<double>& nutritionaldelivery, vector<double>& healthRiskFactors, int& errorFlag) =>
    length_error: cropAreas.size() does not match allCrops.size()

In [46]:
print(cropAreas.size())

11


In [30]:
end = time.time()
print("Run time = {}".format(end - start))

Run time = 2.7320148944854736


In [31]:
print(greenhouseGasEmissions)
print(nLeach)
print(profit)
print(production)

c_double(0.0)
c_double(0.0)
c_double(0.0)
c_double(0.0)


In [17]:
print([i for i in pesticideImpacts])
print([i for i in nutritionaldelivery])
print([i for i in healthRiskFactors])

[1347544556.2169046, 5128180949.047542, 8252271088.746155, 7388689243.319229, 16538141985.144703]
[0.5775431177624832, 0.04107877534263179, 0.01079043833910764, 0.13989707303139862, 0.00824392534901857, 0.07596251832187029, 0.010851330386324876, 0.02459845447397132, 0.11103436699319359]
[]
